## 1. Imports & Data/Model Load

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import tensorflow as tf

In [ ]:
X = np.load('X.npy')  # full or subset
Y = np.load('Y.npy')
X = X[..., np.newaxis]
model = tf.keras.models.load_model('eeg_personid_model.h5')
print('Data:', X.shape, 'Labels:', Y.shape)

## 2. Example Spectrograms
Plot a few random spectrogram windows.

In [ ]:
import random
indices = random.sample(range(len(X)), min(5, len(X)))
plt.figure(figsize=(12,6))
for i, idx in enumerate(indices):
    plt.subplot(1, len(indices), i+1)
    plt.imshow(X[idx].squeeze(), aspect='auto', origin='lower')
    plt.title(f'Subject {Y[idx]+1}')
    plt.axis('off')
plt.tight_layout(); plt.show()

## 3. Feature Embeddings via Penultimate Layer

In [ ]:
feature_extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)
features = feature_extractor.predict(X)
print('Features shape:', features.shape)

## 4. t-SNE Projection
If dataset is large, sample a subset for speed.

In [ ]:
subset_features = features
subset_labels = Y
if subset_features.shape[0] > 2000:
    sel = np.random.choice(subset_features.shape[0], 2000, replace=False)
    subset_features = subset_features[sel]
    subset_labels = subset_labels[sel]
tsne = TSNE(n_components=2, random_state=42)
proj = tsne.fit_transform(subset_features)
plt.figure(figsize=(8,6))
plt.scatter(proj[:,0], proj[:,1], s=6, c=subset_labels, cmap='tab20', alpha=0.7)
plt.title('t-SNE of Feature Embeddings')
plt.colorbar(label='Subject ID')
plt.show()

## 5. Notes
Clusters may indicate discriminability between subjects. Overlap suggests need for richer spatial features or model capacity.